# Bad Characters: <br/>Imperceptible NLP Attacks

See related paper for details.

## Setup
Install dependencies and load pre-treained models.

In [ ]:
%%writefile install-apex.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" \
  --global-option="--deprecated_fused_adam" --global-option="--xentropy" \
  --global-option="--fast_multihead_attn" ./apex

In [ ]:
# Inference is faster with APEX installed
# This installation is optional and takes a while
!sh install-apex.sh

In [ ]:
# Install dependencies
!pip install pyarrow
!pip install fairseq
!pip install sacremoses
!pip install fastBPE
!pip install subword_nmt
!pip install textdistance[extras]
!pip install scipy
!pip install requests

In [ ]:
# Load pre-trained translation model
import torch
en2fr = torch.hub.load('pytorch/fairseq',
                       'transformer.wmt14.en-fr',
                       tokenizer='moses',
                       bpe='subword_nmt').cuda()

## Unknown Characters

Unusual characters, such as zero-width spaces and control sequences, are simply encoded as the `<unk>` character by the FairSeq implementation. This likely generalizes to many other NLP models.

In [ ]:
# Define function for decoding from the source dictionary
def src_decode(sentence):
  res = []
  for idx in sentence:
    res.append(en2fr.src_dict.symbols[idx])
  return ' '.join(res)

In [ ]:
# Define some test encodings
sentence = "This is a test."
psentence = f"This {chr(0x2063)}is a test."
inputv = en2fr.encode(sentence)
outputv = en2fr.generate(inputv)[0]['tokens']
pinputv = en2fr.encode(psentence)
poutputv = en2fr.generate(pinputv)[0]['tokens']

In [ ]:
# Proof that it the encoder just treats invisible chars as an unknown token.
src_decode(pinputv)

'This <unk> is a test . </s>'

## Invisible Characters

Certian Unicode chacacters are almost never visually rendered by design. Conveniently, they can be embedded within strings and copied + pasted on most systems. Most NLP models are not trained against these characters, making them  not present in source language dictionaries. Thus, they typically result in an `<unk>` embedded vector.

In [10]:
# Zero width space
ZWSP = chr(0x200B)
# Zero width joiner
ZWJ = chr(0x200D)
# Zero width non-joiner
ZWNJ = chr(0x200C)

print(f"{ZWSP}{ZWJ}")

​‍


In [ ]:
print(f"This string contains {ZWSP*500}1000{ZWJ*500} invisible characters.")

This string contains ​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​1000‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

## Homoglyphs

The Unicode specification defines several homoglyph documents. The following section retrieves these documents and creates mapping between homoglyph characters.

In [11]:
import requests

confusables = dict()
intentionals = dict()

# Retrieve Unicode Confusable homoglyph characters
conf_resp = requests.get("https://www.unicode.org/Public/security/latest/confusables.txt", stream=True)
for line in conf_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in confusables:
        confusables[line[3]] = []
      confusables[line[3]].append(line[7])

# Retrieve Unicode Intentional homoglyph characters
int_resp = requests.get("https://www.unicode.org/Public/security/latest/intentional.txt", stream=True)
for line in int_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in intentionals:
        intentionals[line[3]] = []
      intentionals[line[3]].append(line[7])

## Reorderings

Unicode Bidirectional (Bidi) Algorithm override characters can be used to render encodeded characters in any order. The following section defines a function which can generate 2^|n| reordered encodings of a given string of length n which are all rendered the same way in any system built on Google's Chromium.

This [site](https://www.soscisurvey.de/tools/view-chars.php) can be used to visualize the underlying encoding of the text.

In [12]:
# Unicode Bidi override characters
PDF = chr(0x202C)
LRE = chr(0x202A)
RLE = chr(0x202B)
LRO = chr(0x202D)
RLO = chr(0x202E)

PDI = chr(0x2069)
LRI = chr(0x2066)
RLI = chr(0x2067)

class Swap():
    """Represents swapped elements in a string of text."""
    def __init__(self, one, two):
        self.one = one
        self.two = two
    
    def __repr__(self):
        return f"Swap({self.one}, {self.two})"

    def __eq__(self, other):
        return self.one == other.one and self.two == other.two

    def __hash__(self):
        return hash((self.one, self.two))

def some(*els):
    """Returns the arguments as a tuple with Nones removed."""
    return tuple(filter(None, tuple(els)))

def swaps(chars: str) -> set:
    """Generates all possible swaps for a string."""
    def pairs(chars, pre=(), suf=()):
        orders = set()
        for i in range(len(chars)-1):
            prefix = pre + tuple(chars[:i])
            suffix = suf + tuple(chars[i+2:])
            swap = Swap(chars[i+1], chars[i])
            pair = some(prefix, swap, suffix)
            orders.add(pair)
            orders.update(pairs(suffix, pre=some(prefix, swap)))
            orders.update(pairs(some(prefix, swap), suf=suffix))
        return orders
    return pairs(chars) | {tuple(chars)}

def unswap(el: tuple) -> str:
    """Reverts a tuple of swaps to the original string."""
    if isinstance(el, str):
        return el
    elif isinstance(el, Swap):
        return unswap((el.two, el.one))
    else:
        res = ""
        for e in el:
            res += unswap(e)
        return res

def uniswap(els):
    res = ""
    for el in els:
        if isinstance(el, Swap):
            res += uniswap([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
        elif isinstance(el, str):
            res += el
        else:
            for subel in el:
                res += uniswap([subel])
    return res

def strings_to_file(file, string):
  with open(file, 'w') as f:
      for swap in swaps(string):
          uni = uniswap(swap)
          print(uni, file=f)

def print_strings(string):
  for swap in swaps(string):
    uni = uniswap(swap)
    print(uni)

In [ ]:
# Prove number of permutations = 2^|n|-1
string = "A test string."
assert(len(swaps(string))) == 2**(len(string)-1)

In [ ]:
# Print strings
print_strings("Test")

Test
‭⁦‮⁦e⁩⁦T⁩‬⁩‬‭⁦‮⁦t⁩⁦s⁩‬⁩‬
‭⁦‮⁦e⁩⁦T⁩‬⁩‬st
‭⁦‮⁦‭⁦‮⁦s⁩⁦e⁩‬⁩‬⁩⁦T⁩‬⁩‬t
‭⁦‮⁦‭⁦‮⁦t⁩⁦s⁩‬⁩‬⁩⁦‭⁦‮⁦e⁩⁦T⁩‬⁩‬⁩‬⁩‬
Te‭⁦‮⁦t⁩⁦s⁩‬⁩‬
‭⁦‮⁦‭⁦‮⁦t⁩⁦s⁩‬⁩‬⁩⁦Te⁩‬⁩‬
T‭⁦‮⁦s⁩⁦e⁩‬⁩‬t


In [ ]:
# Save strings to file
strings_to_file('reordering_example.txt', "This is a test.")

## Deletions

Unicode control characters used for deleting text can be encoded into strings. Upon rendering, these control characters are actioned and the appropriate surrounding text is not rendered. Yet, NLP models generally still "see" the surrounding text.

In [13]:
# Backspace character
BKSP = chr(0x8)
# Delete character
DEL = chr(0x7F)
# Carriage return character
CR = chr(0xD)

print(f"{CR}{BKSP}{DEL}")




In [ ]:
# Deletion charaters are interesting, as surrounding text isn't visually
# rendered, but is "seen" by the model
inp = f"father{BKSP*6}"
res = en2fr.encode(inp)
print(f"Input Rendering: '{inp}'")
print(f"Input Encoding: '{src_decode(res)}'")
print(f"Output: '{en2fr.translate(inp)}'")

Input Rendering: 'father'
Input Encoding: 'father </s>'
Output: 'père'


In [ ]:
# The carriage return character returns the beginning of the line and
# ovewrites it
inp = f"grand{CR}father"
res = en2fr.encode(inp)
print(f"Input Rendering:\n{inp}")
print(f"Input Encoding: '{src_decode(res)}'")
print(f"Output: '{en2fr.translate(inp)}'")

Input Rendering:
father
Input Encoding: 'grand father </s>'
Output: 'grand-père'


## Untargeted Integrity Attacks

The performance of various NLP models can be degraded through the use of invisible character, homoglyph, reordering, and deletion attacks. The most effective attacks can be found, independent of the underlying model, using a genetic algorithm.

### Attack Setup

Each attack will be defined as an object and set of contstraints over which a genetic algorithm (differential evolution) will optimize. For these attacks, the visual representation of the input is fixed and the aim of the attack is to determine the imperceptible perturbation for which the supplied model's output will be maximally distant from the output of the unperturbed input.

Each attack will be derived from the following Objective abstract class.

In [14]:
from abc import ABC
from typing import List, Tuple, Callable, Dict
from fairseq.hub_utils import GeneratorHubInterface
from scipy.optimize import NonlinearConstraint, differential_evolution
from textdistance import levenshtein
import numpy as np


class Objective(ABC):
  """ Abstract class representing objectives for scipy's genetic algorithms."""

  def __init__(self, model: GeneratorHubInterface, input: str, max_perturbs: int, distance: Callable[[str,str],int]):
    if not model:
      raise ValueError("Must supply model.")
    if not input:
      raise ValueError("Must supply input.")

    self.model: GeneratorHubInterface = model
    self.input: str = input
    self.max_perturbs: int = max_perturbs
    self.distance: Callable[[str,str],int] = distance
    self.output = self.model.translate(self.input)

  def objective(self) -> Callable[[List[float]], float]:
    def _objective(perturbations: List[float]) -> float:
      candidate: str = self.candidate(perturbations)
      translation: str = self.model.translate(candidate)
      return -self.distance(self.output, translation)
    return _objective

  def differential_evolution(self, print_result=True, verbose=True, maxiter=60, popsize=32, polish=False) -> str:
    result = differential_evolution(self.objective(), self.bounds(),
                                    disp=verbose, maxiter=maxiter,
                                    popsize=popsize, polish=polish)
    candidate = self.candidate(result.x)
    if (print_result):
      print(f"Result: {candidate}")
      print(f"Result Distance: {result.fun}")
      print(f"Perturbation Encoding: {result.x}")
      print(f"Input Translation: {self.output}")
      print(f"Result Translation: {self.model.translate(candidate)}")
    return candidate

  def bounds(self) -> List[Tuple[float, float]]:
    raise NotImplementedError()

  def candidate(self, perturbations: List[float]) -> str:
    raise NotImplementedError()


def natural(x: float) -> int:
    """Rounds float to the nearest natural number (positive int)"""
    return max(0, round(float(x)))

### Machine Translation Attacks

These attacks are against the fariseq EN-FR translation model.

#### Invisible Character Attack

The following attack injects the invisible character supplied to create the maximally effective imperceptible perturbation against the supplied visual input and distance metric.

In [15]:
class InvisibleCharacterObjective(Objective):
  """Class representing an Objective which injects invisible characters."""

  def __init__(self, model: GeneratorHubInterface, input: str, max_perturbs: int = 25, invisible_chrs: List[str] = [ZWJ,ZWSP,ZWNJ], distance: Callable[[str,str],int] = levenshtein.distance, **kwargs):
    super().__init__(model, input, max_perturbs, distance)
    self.invisible_chrs: List[str] = invisible_chrs

  def bounds(self) -> List[Tuple[float, float]]:
    return [(0,len(self.invisible_chrs)-1), (-1, len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      inp_index = natural(perturbations[i+1])
      if inp_index >= 0:
        inv_char = self.invisible_chrs[natural(perturbations[i])]
        candidate = candidate[:inp_index] + [inv_char] + candidate[inp_index:]
    return ''.join(candidate)

In [ ]:
# Execute Example Invisible Characters Attack
inv_obj = InvisibleCharacterObjective(en2fr, 'Spectacular Wingsuit Jump Over Bogota', max_perturbs=2)
inv_result = inv_obj.differential_evolution(maxiter=40, popsize=32)

#### Homoglyph Attack

This attack replaces characters with homoglyphs to create the maximally effective imperceptible perturbation against the supplied visual input and distance metric.

In [16]:
class HomoglyphObjective(Objective):

  def __init__(self, model: GeneratorHubInterface, input: str, max_perturbs=None, distance: Callable[[str,str],int] = levenshtein.distance, homoglyphs: Dict[str,List[str]] = intentionals, **kwargs):
    super().__init__(model, input, max_perturbs, distance)
    if not self.max_perturbs:
      self.max_perturbs = len(self.input)
    self.homoglyphs = homoglyphs
    self.glyph_map = []
    for i, char in enumerate(self.input):
      if char in self.homoglyphs:
        charmap = self.homoglyphs[char]
        charmap = list(zip([i] * len(charmap), charmap))
        self.glyph_map.extend(charmap)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1, len(self.glyph_map)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]  
    for perturb in map(natural, perturbations):
      if perturb >= 0:
        i, char = self.glyph_map[perturb]
        candidate[i] = char
    return ''.join(candidate)

In [ ]:
# Execute Example Homoglyph Attack
homo_obj = HomoglyphObjective(en2fr, 'Spectacular Wingsuit Jump Over Bogota', max_perturbs=3)
homo_result = homo_obj.differential_evolution(maxiter=5)

#### Reordering Attack

This attack reorders characters using Unicode Bidi overrides to create the maximally effective imperceptible perturbation against the supplied visual input and distance metric.

The reordering patterns used in this attack were designed to be effective against the Bidi implementation used in the Chromium text rendering engine.

In [17]:
class ReorderObjective(Objective):

  def __init__(self, model: GeneratorHubInterface, input: str, max_perturbs: int = 50, distance: Callable[[str,str],int] = levenshtein.distance, **kwargs):
    super().__init__(model, input, max_perturbs, distance)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    def swaps(els) -> str:
      res = ""
      for el in els:
          if isinstance(el, Swap):
              res += swaps([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
          elif isinstance(el, str):
              res += el
          else:
              for subel in el:
                  res += swaps([subel])
      return res

    _candidate = [char for char in self.input]
    for perturb in map(natural, perturbations):
      if perturb >= 0 and len(_candidate) >= 2:
        perturb = min(perturb, len(_candidate) - 2)
        _candidate = _candidate[:perturb] + [Swap(_candidate[perturb+1], _candidate[perturb])] + _candidate[perturb+2:]

    return swaps(_candidate)

In [ ]:
# Execute Example Reordering Attack
reorder_obj = ReorderObjective(en2fr, 'Spectacular Wingsuit Jump Over Bogota', max_perturbs=2)
reorder_result = reorder_obj.differential_evolution(maxiter=5, popsize=5)

differential_evolution step 1: f(x)= -78
differential_evolution step 2: f(x)= -80
differential_evolution step 3: f(x)= -86
differential_evolution step 4: f(x)= -96
differential_evolution step 5: f(x)= -140
Result: Spectacular Wingsuit Jump Over ‭⁦‮⁦o⁩⁦B⁩‬⁩‬go‭⁦‮⁦a⁩⁦t⁩‬⁩‬
Result Distance: -140
Perturbation Encoding: [34.6174655  31.19274238]
Input Translation: Spectaculaire combinaison pour les ailes sauter au-dessus de Bogota
Result Translation: Saut en combinaison à ailes spectaculaire Saut au-dessus d'une combinaison à voilure spectaculaire Saut au-dessus d'une combinaison à voilure spectaculaire Saut au-dessus d'une combinaison à voilure


#### Deletion Attack

This attack inserts Unicode deletion control characters followed by a supplied character to be deleted to create the maximally effective imperceptible perturbation against the supplied visual input and distance metric.

In [18]:
class DeletionObjective(Objective):
  """Class representing an Objective which injects deletion control characters."""

  def __init__(self, model: GeneratorHubInterface, input: str, max_perturbs: int = 100, distance: Callable[[str,str],int] = levenshtein.distance, del_chr: str = BKSP, ins_chr_min: str = '!', ins_chr_max: str = '~', **kwargs):
    super().__init__(model, input, max_perturbs, distance)
    self.del_chr: str = del_chr
    self.ins_chr_min: str = ins_chr_min
    self.ins_chr_max: str = ins_chr_max

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1), (ord(self.ins_chr_min),ord(self.ins_chr_max))] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      idx = natural(perturbations[i])
      char = chr(natural(perturbations[i+1]))
      candidate = candidate[:idx] + [char, self.del_chr] + candidate[idx:]
      for j in range(i,len(perturbations), 2):
        perturbations[j] += 2
    return ''.join(candidate)

In [ ]:
# Execute Example Reordering Attack
deletion_obj = DeletionObjective(en2fr, "This is a test", max_perturbs=2)
deletion_result = deletion_obj.differential_evolution(maxiter=10, popsize=10)

differential_evolution step 1: f(x)= -27
differential_evolution step 2: f(x)= -28
differential_evolution step 3: f(x)= -28
differential_evolution step 4: f(x)= -28
differential_evolution step 5: f(x)= -28
differential_evolution step 6: f(x)= -28
differential_evolution step 7: f(x)= -28
differential_evolution step 8: f(x)= -28
differential_evolution step 9: f(x)= -28
Result: Thais is a taest
Result Distance: -28.0
Perturbation Encoding: [12.75208361  3.78647273]
Input Translation: Il s'agit d'un test
Result Translation: Les Thaïlandais sont les plus avides


#### Attack Performance

In this section we plot the results of each of the attacks over pertrubations of fixed sizes.

##### Experiment Setup

In [ ]:
# Download EN-FR Test Data
!wget -c http://statmt.org/wmt14/test-full.tgz -O - | tar -xz
!mv test-full/newstest2014-fren-src.en.sgm .
!mv test-full/newstest2014-fren-ref.fr.sgm .
!rm -rf test-full/
!pip install beautifulsoup4

In [29]:
# Build source and target mappings for BLEU scoring
from bs4 import BeautifulSoup

with open('newstest2014-fren-src.en.sgm', 'r') as f:
  source_doc = BeautifulSoup(f, 'html.parser')

with open('newstest2014-fren-ref.fr.sgm', 'r') as f:
  target_doc = BeautifulSoup(f, 'html.parser')

source = dict()
target = dict()

for doc in source_doc.find_all('doc'):
  source[str(doc['docid'])] = dict()
  for seg in doc.find_all('seg'):
    if len(str(seg.string)) < 50:
      source[str(doc['docid'])][str(seg['id'])] = str(seg.string)

for doc in target_doc.find_all('doc'):
  target[str(doc['docid'])] = dict()
  for seg in doc.find_all('seg'):
    target[str(doc['docid'])][str(seg['id'])] = str(seg.string)

In [31]:
# Experiemnt for testing objectives within sliding perturbation bounds
from tqdm.auto import tqdm, trange
import pickle

def experiment(model, objective, source, perturbs, min_perturb, max_perturb, file, maxiter, popsize):
  for i in trange(min_perturb, max_perturb, desc="Perturbations"):
    perturbs[str(i)] = dict()
    for docid, doc in tqdm(source.items(), leave=False, desc="Document"):
      perturbs[str(i)][docid] = dict()
      for segid, seg in tqdm(doc.items(), leave=False, desc="Sequence"):
        perturbs[str(i)][docid][segid] = objective(en2fr, seg, max_perturbs=i).differential_evolution(print_result=False, verbose=False, maxiter=maxiter, popsize=popsize)
        with open(file, 'wb') as f:
          pickle.dump(perturbs, f)

##### Invisible Character Experiment

In [ ]:
# Invisible Character Experiment

# Only consider a subset (65 examples) of the source
source_small = { '1016-latimes': source['1016-latimes'], '2327-dailymail.co.uk': source['2327-dailymail.co.uk'] }
perturbs = { '0': source_small }

experiment(en2fr, InvisibleCharacterObjective, source_small, perturbs, 1, 6, 'invisible_chars.pkl', 5, 16)

##### Homoglyph Experiment

In [ ]:
# Homoglyph Experiment

experiment(en2fr, HomoglyphObjective, source_small, perturbs, 1, 6, 'homoglyphs.pkl', 3, 16)

##### Reordering Experiment

In [ ]:
# Reordering Experiment

experiment(en2fr, ReorderObjective, source_small, perturbs, 1, 6, 'reorder.pkl', 3, 16)

##### Deletion Experiment

In [ ]:
# Deletion Experiment

experiment(en2fr, DeletionObjective, source_small, perturbs, 1, 6, 'deletion.pkl', 3, 16)

### MNLI Attacks

#### Attack Setup

In [ ]:
!wget https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip
!unzip multinli_1.0.zip
!rm -rf __MACOSX/

In [ ]:
import json
with open('multinli_1.0/multinli_1.0_dev_matched.jsonl', 'r') as f:
  mnli_test = [json.loads(jline) for jline in f.readlines()]

In [ ]:
# Load pre-trained translation model
import torch
mnli = torch.hub.load('pytorch/fairseq',
                       'roberta.large.mnli').eval().cuda()
label_map = {'contradiction': 0, 'neutral': 1, 'entailment': 2}

In [ ]:
class MnliObjective():

  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, max_perturbs: int):
    if not model:
      raise ValueError("Must supply model.")
    if not input:
      raise ValueError("Must supply input.")
    if not hypothesis:
      raise ValueError("Must supply hypothesis.")
    if label == None:
      raise ValueError("Must supply label.")
    self.model: GeneratorHubInterface = model
    self.input: str = input
    self.hypothesis: str = hypothesis
    self.label: int = label
    self.max_perturbs: int = max_perturbs

  def objective(self) -> Callable[[List[float]], float]:
    def _objective(perturbations: List[float]) -> float:
      candidate: str = self.candidate(perturbations)
      tokens = self.model.encode(candidate, self.hypothesis)
      predict = self.model.predict('mnli', tokens)
      if predict.argmax() != self.label:
        return -np.inf
      else:
        return predict.cpu().detach().numpy()[0][self.label]
    return _objective

  def differential_evolution(self, print_result=True, verbose=True, maxiter=3, popsize=32, polish=False) -> str:
    result = differential_evolution(self.objective(), self.bounds(),
                                    disp=verbose, maxiter=maxiter,
                                    popsize=popsize, polish=polish)
    candidate = self.candidate(result.x)
    if (print_result):
      print(f"Result: {candidate}")
      print(f"Correct Label Prediction: {result.fun}")
      print(f"Perturbation Encoding: {result.x}")
    return candidate

#### Invisible Character Attack

In [ ]:
class InvisibleCharacterMnliObjective(MnliObjective, InvisibleCharacterObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, max_perturbs: int = 10, invisible_chrs: List[str] = [ZWJ,ZWSP,ZWNJ], **kwargs):
    super().__init__(model, input, hypothesis, label, max_perturbs)
    self.invisible_chrs = invisible_chrs

#### Homoglyph Attack

In [ ]:
class HomoglyphMnliObjective(MnliObjective, HomoglyphObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, max_perturbs: int = 10, homoglyphs: Dict[str,List[str]] = intentionals, **kwargs):
    super().__init__(model, input, hypothesis, label, max_perturbs)
    self.homoglyphs = homoglyphs
    self.glyph_map = []
    for i, char in enumerate(self.input):
      if char in self.homoglyphs:
        charmap = self.homoglyphs[char]
        charmap = list(zip([i] * len(charmap), charmap))
        self.glyph_map.extend(charmap)

#### Reordering Attack

In [ ]:
class ReorderMnliObjective(MnliObjective, ReorderObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, max_perturbs: int = 10, **kwargs):
    super().__init__(model, input, hypothesis, label, max_perturbs)

#### Deletion Attack

In [ ]:
class DeletionMnliObjective(MnliObjective, DeletionObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, max_perturbs: int = 10, del_chr: str = BKSP, ins_chr_min: str = '!', ins_chr_max: str = '~', **kwargs):
    super().__init__(model, input, hypothesis, label, max_perturbs)
    self.del_chr: str = del_chr
    self.ins_chr_min: str = ins_chr_min
    self.ins_chr_max: str = ins_chr_max

#### Attack Performance

##### Experiment Setup

In [ ]:
def mnli_experiment(model, objective, data, file, min_budget, max_budget, maxiter, popsize):
  perturbs = { '0': data }
  for budget in trange(min_budget, max_budget):
    perturbs[str(budget)] = dict()
    for test in tqdm(data, leave=False):
      obj = objective(mnli, test['sentence1'], test['sentence2'], label_map[test['gold_label']])
      example = obj.differential_evolution(print_result=False, verbose=False, maxiter=maxiter, popsize=popsize)
      perturbs[str(budget)][test['pairID']] = example
      with open(file, 'wb') as f:
          pickle.dump(perturbs, f)

##### Invisible Character Experiment

In [ ]:
mnli_experiment(mnli, InvisibleCharacterMnliObjective, mnli_test[:20], "mnli_invisible_chars.pkl", 1, 6, 3, 16)

##### Homoglyph Experiment

In [ ]:
mnli_experiment(mnli, HomoglyphMnliObjective, mnli_test[:20], "mnli_homoglyphs.pkl", 1, 6, 3, 16)

##### Reordering Experiment

In [ ]:
mnli_experiment(mnli, ReorderMnliObjective, mnli_test[:20], "mnli_reorder.pkl", 1, 6, 3, 16)

##### Deletion Experiment

In [ ]:
mnli_experiment(mnli, DeletionMnliObjective, mnli_test[:20], "mnli_deletion.pkl", 1, 6, 3, 16)

### IBM Toxic Content Classifier Attack

Untargeted integrity attacks on the open source IBM Toxic Content filter. Details of the model are available [here](https://github.com/IBM/MAX-Toxic-Comment-Classifier).

####Attack Setup

In [ ]:
%%writefile install-apex.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" \
  --global-option="--deprecated_fused_adam" --global-option="--xentropy" \
  --global-option="--fast_multihead_attn" ./apex

In [ ]:
!sh install-apex.sh
!rm -rf assets/
!rm -rf toxic/
!mkdir assets
!wget https://max-cdn.cdn.appdomain.cloud/max-toxic-comment-classifier/1.0.0/assets.tar.gz --output-document=assets/assets.tar.gz
!tar -x -C assets/ -f assets/assets.tar.gz -v
!rm assets/assets.tar.gz
!git clone https://github.com/IBM/MAX-Toxic-Comment-Classifier.git
!mv MAX-Toxic-Comment-Classifier toxic
!sed -i 's/==.*//g' toxic/requirements.txt
!pip install -r toxic/requirements.txt
!pip install maxfw
!sed -i 's/from config/from ..config/g' toxic/core/model.py
!sed -i 's/from core\./from ./g' toxic/core/model.py

In [ ]:
from toxic.core.model import ModelWrapper
from logging import getLogger, WARNING
import torch
getLogger().setLevel(WARNING)
toxic = ModelWrapper()
if torch.cuda.is_available():
  toxic.model.cuda()
  toxic.device = torch.device("cuda")

In [6]:
class ToxicObjective(ABC):
  """ Toxic content filter objective. """

  def __init__(self, model, input: str, max_perturbs: int):
    if not model:
      raise ValueError("Must supply model.")
    if not input:
      raise ValueError("Must supply input.")

    self.model = model
    self.input: str = input
    self.max_perturbs: int = max_perturbs
    self.output = self.model.predict([self.input])[0]

  def objective(self) -> Callable[[List[float]], float]:
    def _objective(perturbations: List[float]) -> float:
      candidate: str = self.candidate(perturbations)
      predicts: str = self.model.predict([candidate])[0]
      return sum(predicts.values())
    return _objective

  def differential_evolution(self, print_result=True, verbose=True, maxiter=60, popsize=32, polish=False) -> str:
    result = differential_evolution(self.objective(), self.bounds(),
                                    disp=verbose, maxiter=maxiter,
                                    popsize=popsize, polish=polish)
    candidate = self.candidate(result.x)
    if (print_result):
      print(f"Result: {candidate}")
      print(f"Result Predictions: {self.model.predict([candidate])[0]}")
      print(f"Input: {self.input}")
      print(f"Input Predictions: {self.output}")
    return candidate

  def bounds(self) -> List[Tuple[float, float]]:
    raise NotImplementedError()

  def candidate(self, perturbations: List[float]) -> str:
    raise NotImplementedError()

####Invisible Character Attack

In [ ]:
class InvisibleToxicObjective(ToxicObjective):
  """Class representing a Toxic Objective which injects invisible characters."""

  def __init__(self, model, input: str, max_perturbs: int = 25, invisible_chrs: List[str] = [ZWJ,ZWSP,ZWNJ], **kwargs):
    super().__init__(model, input, max_perturbs)
    self.invisible_chrs: List[str] = invisible_chrs

  def bounds(self) -> List[Tuple[float, float]]:
    return [(0,len(self.invisible_chrs)-1), (-1, len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      inp_index = natural(perturbations[i+1])
      if inp_index >= 0:
        inv_char = self.invisible_chrs[natural(perturbations[i])]
        candidate = candidate[:inp_index] + [inv_char] + candidate[inp_index:]
    return ''.join(candidate)

####Homoglyph Attack

In [ ]:
class HomoglyphToxicObjective(ToxicObjective):
  """Class representing a Toxic Objective which injects homoglyphs."""

  def __init__(self, model, input: str, max_perturbs: int = 25, homoglyphs: Dict[str,List[str]] = intentionals, **kwargs):
    super().__init__(model, input, max_perturbs)
    self.homoglyphs = homoglyphs
    self.glyph_map = []
    for i, char in enumerate(self.input):
      if char in self.homoglyphs:
        charmap = self.homoglyphs[char]
        charmap = list(zip([i] * len(charmap), charmap))
        self.glyph_map.extend(charmap)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1, len(self.glyph_map)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]  
    for perturb in map(natural, perturbations):
      if perturb >= 0:
        i, char = self.glyph_map[perturb]
        candidate[i] = char
    return ''.join(candidate)

####Reordering Attack

In [ ]:
class ReorderToxicObjective(ToxicObjective):
  """Class representing a Toxic Objective which injects homoglyphs."""

  def __init__(self, model, input: str, max_perturbs: int = 25, **kwargs):
    super().__init__(model, input, max_perturbs)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    def swaps(els) -> str:
      res = ""
      for el in els:
          if isinstance(el, Swap):
              res += swaps([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
          elif isinstance(el, str):
              res += el
          else:
              for subel in el:
                  res += swaps([subel])
      return res

    _candidate = [char for char in self.input]
    for perturb in map(natural, perturbations):
      if perturb >= 0 and len(_candidate) >= 2:
        perturb = min(perturb, len(_candidate) - 2)
        _candidate = _candidate[:perturb] + [Swap(_candidate[perturb+1], _candidate[perturb])] + _candidate[perturb+2:]

    return swaps(_candidate)

####Deletion Attack

In [ ]:
class DeletionToxicObjective(ToxicObjective):
  """Class representing a Toxic Objective which injects homoglyphs."""

  def __init__(self, model, input: str, max_perturbs: int = 25, del_chr: str = BKSP, ins_chr_min: str = '!', ins_chr_max: str = '~', **kwargs):
    super().__init__(model, input, max_perturbs)
    self.del_chr = del_chr
    self.ins_chr_min: str = ins_chr_min
    self.ins_chr_max: str = ins_chr_max

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1), (ord(self.ins_chr_min),ord(self.ins_chr_max))] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      idx = natural(perturbations[i])
      char = chr(natural(perturbations[i+1]))
      candidate = candidate[:idx] + [char, self.del_chr] + candidate[idx:]
      for j in range(i,len(perturbations), 2):
        perturbations[j] += 2
    return ''.join(candidate)

####Attack Performance

#####Experiment Setup

In [ ]:
# Data set details: https://meta.wikimedia.org/wiki/Research:Detox/Data_Release
!wget https://ndownloader.figshare.com/files/7394542 -O toxicity_annotated_comments.tsv
!wget https://ndownloader.figshare.com/files/7394539 -O toxicity_annotations.tsv

In [ ]:
# Load Test Data
import pandas as pd

# load tsv files
comments = pd.read_csv('toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('toxicity_annotations.tsv',  sep = '\t')
# labels a comment as toxic if the majority of annoatators did so
labels = annotations.groupby('rev_id')['toxicity'].mean() > 0.5
# join labels and comments
comments['toxicity'] = labels
# remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
test_comments = comments.query("split=='test'").query("toxicity==True")
examples = test_comments.reset_index().to_dict('records')

In [ ]:
def max_toxic_experiment(objective, model, file, min_budget, max_budget, examples, maxiter, popsize):
  perturbs = { '0': examples }
  with tqdm(total=len(examples)*(max_budget-min_budget+1), desc="Adv. Examples") as pbar:
    for budget in range(min_budget, max_budget+1):
      perturbs[str(budget)] = dict()
      for example in examples:
        obj = objective(model, example['comment'], example['toxicity'], budget)
        result = obj.differential_evolution(verbose=False, print_result=False, maxiter=maxiter, popsize=popsize)
        perturbs[str(budget)][example['rev_id']] = result
        with open(file, 'wb') as f:
          pickle.dump(perturbs, f)
        pbar.update(1)

#####Invisible Character Experiment

In [ ]:
max_toxic_experiment(InvisibleToxicObjective, toxic, 'maxtoxic_invisibles_small.pkl',
                     1, 5, examples[:20], 3, 32)

#####Homoglyph Experiment

In [ ]:
max_toxic_experiment(HomoglyphToxicObjective, toxic, 'maxtoxic_homoglyphs_small.pkl',
                     1, 5, examples[:20], 3, 32)

#####Reordering Experiment

In [ ]:
max_toxic_experiment(ReorderToxicObjective, toxic, 'maxtoxic_reorderings_small.pkl',
                     1, 5, examples[:20], 3, 32)

#####Deletion Experiment

In [ ]:
max_toxic_experiment(DeletionToxicObjective, toxic, 'maxtoxic_deletions_small.pkl',
                     1, 5, examples[:20], 3, 32)

###Perspective API Toxic Content Classifier Attack

Perspective API is a toxic content classifier offered by Google. Details are available [here](https://www.perspectiveapi.com/). You will need to register and get an API key to run these experiments.

####Attack Setup

In [ ]:
import time

class ToxicPerspectiveObjective(ABC):
  """Class representing attacks against Google's toxic language Perspective API."""

  def __init__(self, client, input: str, max_perturbs: int, annotated_toxic: bool):
    if not client:
      raise ValueError("Must supply client.")
    if not input:
      raise ValueError("Must supply input.")

    self.client = client
    self.input: str = input
    self.max_perturbs: int = max_perturbs
    self.annotated_toxic = annotated_toxic
    self.last_request = time.time()
    # Perspective API rate limit in seconds
    self.rate_limit = 0.105
    self.output = self.request(input)

  def request(self, candidate: str):
    delta = time.time() - self.last_request
    if delta < self.rate_limit:
      time.sleep(self.rate_limit - delta)
    self.last_request = time.time()
    analyze_request = {
        'comment': { 'text': candidate },
        'requestedAttributes': {'TOXICITY': {}},
        'languages': 'en',
      }
    response = self.client.comments().analyze(body=analyze_request).execute()
    return response['attributeScores']['TOXICITY']['summaryScore']['value']

  def objective(self) -> Callable[[List[float]], float]:
    def _objective(perturbations: List[float]) -> float:
      candidate: str = self.candidate(perturbations)
      return self.request(candidate)
    return _objective
    
  def differential_evolution(self, print_result=True, verbose=True, maxiter=60, popsize=32, polish=False) -> str:
    start = time.process_time()
    result = differential_evolution(self.objective(), self.bounds(),
                                    disp=verbose, maxiter=maxiter,
                                    popsize=popsize, polish=polish)
    end = time.process_time()
    candidate = self.candidate(result.x)
    adv_output = self.request(candidate)
    if (print_result):
      print(f"Result: {candidate}")
      print(f"Result Predictions: {adv_output}")
      print(f"Input: {self.input}")
      print(f"Input Predictions: {self.output}")
    return candidate

  def bounds(self) -> List[Tuple[float, float]]:
    raise NotImplementedError()

  def candidate(self, perturbations: List[float]) -> str:
    raise NotImplementedError()

In [ ]:
from googleapiclient.discovery_cache.base import Cache
from googleapiclient import discovery
from getpass import getpass
import json


class MemoryCache(Cache):
    _CACHE = {}

    def get(self, url):
        return MemoryCache._CACHE.get(url)

    def set(self, url, content):
        MemoryCache._CACHE[url] = content


perspective = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=getpass("Perspective API Key: "),
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  cache=MemoryCache()
)

####Invisisble Character Attack

In [ ]:
class InvisibleToxicPerspectiveObjective(ToxicPerspectiveObjective):
  """Class representing a Toxic Perspective API Objective which injects invisible characters."""

  def __init__(self, client, input: str, annotated_toxic: bool, max_perturbs: int = 25, invisible_chrs: List[str] = [ZWJ,ZWSP,ZWNJ], **kwargs):
    super().__init__(client, input, max_perturbs, annotated_toxic)
    self.invisible_chrs: List[str] = invisible_chrs

  def bounds(self) -> List[Tuple[float, float]]:
    return [(0,len(self.invisible_chrs)-1), (-1, len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      inp_index = natural(perturbations[i+1])
      if inp_index >= 0:
        inv_char = self.invisible_chrs[natural(perturbations[i])]
        candidate = candidate[:inp_index] + [inv_char] + candidate[inp_index:]
    return ''.join(candidate)

####Homoglyph Attack

In [ ]:
class HomoglyphToxicPerspectiveObjective(ToxicPerspectiveObjective):
  """Class representing a Toxic Perspective API Objective which injects homoglyphs."""

  def __init__(self, client, input: str, annotated_toxic: bool, max_perturbs: int = 25, homoglyphs: Dict[str,List[str]] = intentionals, **kwargs):
    super().__init__(client, input, max_perturbs, annotated_toxic)
    self.homoglyphs = homoglyphs
    self.glyph_map = []
    for i, char in enumerate(self.input):
      if char in self.homoglyphs:
        charmap = self.homoglyphs[char]
        charmap = list(zip([i] * len(charmap), charmap))
        self.glyph_map.extend(charmap)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1, len(self.glyph_map)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]  
    for perturb in map(natural, perturbations):
      if perturb >= 0:
        i, char = self.glyph_map[perturb]
        candidate[i] = char
    return ''.join(candidate)

####Reordering Attack

In [ ]:
class ReorderToxicPerspectiveObjective(ToxicPerspectiveObjective):
  """Class representing a Toxic Perspective API Objective which injects reorderings."""

  def __init__(self, client, input: str, annotated_toxic: bool, max_perturbs: int = 25, **kwargs):
    super().__init__(client, input, max_perturbs, annotated_toxic)

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1)] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    def swaps(els) -> str:
      res = ""
      for el in els:
          if isinstance(el, Swap):
              res += swaps([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
          elif isinstance(el, str):
              res += el
          else:
              for subel in el:
                  res += swaps([subel])
      return res

    _candidate = [char for char in self.input]
    for perturb in map(natural, perturbations):
      if perturb >= 0 and len(_candidate) >= 2:
        perturb = min(perturb, len(_candidate) - 2)
        _candidate = _candidate[:perturb] + [Swap(_candidate[perturb+1], _candidate[perturb])] + _candidate[perturb+2:]

    return swaps(_candidate)

####Deletion Attack

In [ ]:
class DeletionToxicPerspectiveObjective(ToxicPerspectiveObjective):
  """Class representing a Toxic Perspective API Objective which injects deletions."""

  def __init__(self, client, input: str, annotated_toxic: bool, max_perturbs: int = 25, del_chr: str = BKSP, ins_chr_min: str = '!', ins_chr_max: str = '~', **kwargs):
    super().__init__(client, input, max_perturbs, annotated_toxic)
    self.del_chr = del_chr
    self.ins_chr_min: str = ins_chr_min
    self.ins_chr_max: str = ins_chr_max

  def bounds(self) -> List[Tuple[float, float]]:
    return [(-1,len(self.input)-1), (ord(self.ins_chr_min),ord(self.ins_chr_max))] * self.max_perturbs

  def candidate(self, perturbations: List[float]) -> str:
    candidate = [char for char in self.input]
    for i in range(0, len(perturbations), 2):
      idx = natural(perturbations[i])
      char = chr(natural(perturbations[i+1]))
      candidate = candidate[:idx] + [char, self.del_chr] + candidate[idx:]
      for j in range(i,len(perturbations), 2):
        perturbations[j] += 2
    return ''.join(candidate)

####Attack Performance

#####Experiment Setup

In [ ]:
def perspective_experiment(objective, client, file, min_budget, max_budget, examples, maxiter, popsize):
  perturbs = { '0': examples }
  with tqdm(total=len(examples)*(max_budget-min_budget+1), desc="Adv. Examples") as pbar:
    for budget in range(min_budget, max_budget+1):
      perturbs[str(budget)] = dict()
      for example in examples:
        obj = objective(client, example['comment'], budget)
        result = obj.differential_evolution(verbose=False, print_result=False, maxiter=maxiter, popsize=popsize)
        perturbs[str(budget)][example['rev_id']] = result
        with open(file, 'wb') as f:
          pickle.dump(perturbs, f)
        pbar.update(1)

#####Invisible Character Experiment

In [ ]:
perspective_experiment(InvisibleToxicPerspectiveObjective, perspective, 'perspective_invisibles_small_20.pkl',
                       1, 5, examples[:20], 3, 32)

#####Homoglyph Experiment

In [ ]:
perspective_experiment(HomoglyphToxicPerspectiveObjective, perspective, 'perspective_homoglyphs_small_20.pkl',
                       1, 5, examples[:20], 3, 32)

#####Reordering Experiment

In [ ]:
perspective_experiment(ReorderToxicPerspectiveObjective, perspective, 'perspective_reoderings_small_20.pkl',
                       1, 5, examples[:20], 3, 32)

#####Deletion Experiment

In [ ]:
perspective_experiment(DeletionToxicPerspectiveObjective, perspective, 'perspective_deletions_small_20.pkl',
                       1, 5, examples[:20], 3, 32)

## Targeted Integrity Attacks

Targeted imperceptible perturbation attacks craft imperceptible perturbations for a given input that attempt to prodice a fixed output against a given model.

###MNLI Targeted Attack

These attacks target the MNLI textual entailment classification task in a black box model that does have access to the resulting logits for each class during inference.

####Experiment Setup

In [ ]:
from torch.nn.functional import softmax

class MnliTargetedObjective(MnliObjective):

  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label: int, target: int, max_perturbs: int):
    super().__init__(model, input, hypothesis, label, max_perturbs)
    self.target = target

  def objective(self) -> Callable[[List[float]], float]:
      def _objective(perturbations: List[float]) -> float:
        candidate: str = self.candidate(perturbations)
        tokens = self.model.encode(candidate, self.hypothesis)
        predict = self.model.predict('mnli', tokens)
        return -softmax(predict, dim=1).cpu().detach().numpy()[0][self.target]
      return _objective

  def differential_evolution(self, print_result=True, verbose=True, maxiter=3, popsize=32, polish=False) -> str:
    result = differential_evolution(self.objective(), self.bounds(),
                                    disp=verbose, maxiter=maxiter,
                                    popsize=popsize, polish=polish)
    candidate = self.candidate(result.x)
    if (print_result):
      print(f"Result: {candidate}")
      print(f"Correct Label Prediction: {result.fun}")
      print(f"Perturbation Encoding: {result.x}")
    return candidate

####Invisible Character Attack

In [ ]:
class InvisibleCharacterTargetedMnliObjective(MnliTargetedObjective, InvisibleCharacterObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, target: int, max_perturbs: int = 10, invisible_chrs: List[str] = [ZWJ,ZWSP,ZWNJ], **kwargs):
    super().__init__(model, input, hypothesis, label, target, max_perturbs)
    self.invisible_chrs = invisible_chrs

####Homoglyph Attack

In [ ]:
class HomoglyphTargetedMnliObjective(MnliTargetedObjective, HomoglyphObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, target: int, max_perturbs: int = 10, homoglyphs: Dict[str,List[str]] = intentionals, **kwargs):
    super().__init__(model, input, hypothesis, label, target, max_perturbs)
    self.homoglyphs = homoglyphs
    self.glyph_map = []
    for i, char in enumerate(self.input):
      if char in self.homoglyphs:
        charmap = self.homoglyphs[char]
        charmap = list(zip([i] * len(charmap), charmap))
        self.glyph_map.extend(charmap)

####Reordering Attack

In [ ]:
class ReorderTargetedMnliObjective(MnliTargetedObjective, ReorderObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, target: int, max_perturbs: int = 10, **kwargs):
    super().__init__(model, input, hypothesis, label, target, max_perturbs)

####Deletion Attack

In [ ]:
class DeletionTargetedMnliObjective(MnliTargetedObjective, DeletionObjective):
  
  def __init__(self, model: GeneratorHubInterface, input: str, hypothesis: str, label:int, target: int, max_perturbs: int = 10, del_chr: str = BKSP, ins_chr_min: str = '!', ins_chr_max: str = '~', **kwargs):
    super().__init__(model, input, hypothesis, label, target, max_perturbs)
    self.del_chr = del_chr
    self.ins_chr_min: str = ins_chr_min
    self.ins_chr_max: str = ins_chr_max

####Attack Performance

#####Experiment Setup

In [ ]:
from tqdm.auto import tqdm
from time import process_time
import pickle

def mnli_targeted_experiment(objective, model, label_map, inputs, file, min_budget = 10, max_budget = 10, maxiter = 10, popsize = 32):
  results = { '0': inputs }
  with tqdm(total=len(inputs)*(max_budget-min_budget+1)*len(label_map), desc="Adv. Examples") as pbar:
    for budget in range(min_budget, max_budget+1):
      results[str(budget)] = {}
      for input in inputs:
        results[str(budget)][input['pairID']] = []
        for label in range(len(label_map)):
          obj = objective(model, input['sentence1'], input['sentence2'], label_map[input['gold_label']], label, max_perturbs=budget)
          candidate = obj.differential_evolution(print_result=False, verbose=False, maxiter=maxiter, popsize=popsize)
          results[str(budget)][input['pairID']].append(candidate)
          with open(file, 'wb') as f:
            pickle.dump(results, f)
          pbar.update(1)

#####Invisible Character Experiment

In [ ]:
mnli_targeted_experiment(InvisibleCharacterTargetedMnliObjective, mnli, label_map, mnli_test[:20], "mnli_invisibles_targeted.pkl", maxiter=3)

#####Homoglyph Experiment

In [ ]:
mnli_targeted_experiment(HomoglyphTargetedMnliObjective, mnli, label_map, mnli_test[:20], "mnli_homoglyphs_targeted.pkl", maxiter=3)

#####Redordering Experiment

In [ ]:
mnli_targeted_experiment(ReorderTargetedMnliObjective, mnli, label_map, mnli_test[:20], "mnli_reorderings_targeted.pkl", maxiter=3)

#####Deletion Experiment

In [ ]:
mnli_targeted_experiment(DeletionTargetedMnliObjective, mnli, label_map, mnli_test[:20], "mnli_deletions_targeted.pkl", maxiter=3)

###MNLI (No Logits) Targeted Attack

This attack is identical to the MNLI targeted attack except that the adversary only has access to the predicted class and does not have access to the resulting logits for each class during inference.

####Attack Setup

In [ ]:
class MnliTargetedNoLogitsObjective(MnliTargetedObjective):

  def objective(self) -> Callable[[List[float]], float]:
      def _objective(perturbations: List[float]) -> float:
        candidate: str = self.candidate(perturbations)
        tokens = self.model.encode(candidate, self.hypothesis)
        predict = self.model.predict('mnli', tokens)
        if predict.argmax().item() == self.target:
          return -np.inf
        else:
          return np.inf
      return _objective

####Invisible Character Attack

In [ ]:
class InvisibleCharacterTargetedMnliNoLogitsObjective(MnliTargetedNoLogitsObjective, InvisibleCharacterTargetedMnliObjective):
  pass

####Homoglyph Attack

In [ ]:
class HomoglyphTargetedMnliNoLogitsObjective(MnliTargetedNoLogitsObjective, HomoglyphTargetedMnliObjective):
  pass

####Reordering Attack

In [ ]:
class ReorderTargetedMnliNoLogitsObjective(MnliTargetedNoLogitsObjective, ReorderTargetedMnliObjective):
  pass

####Deletion Attack

In [ ]:
class DeletionTargetedMnliNoLogitsObjective(MnliTargetedNoLogitsObjective, DeletionTargetedMnliObjective):
  pass

####Attack Performance

#####Invisible Character Experiment

In [ ]:
mnli_targeted_experiment(InvisibleCharacterTargetedMnliNoLogitsObjective, mnli, label_map, mnli_test[:20], "mnli_invisibles_targeted_nologits.pkl", maxiter=3)

#####Homoglyph Experiment

In [ ]:
mnli_targeted_experiment(HomoglyphTargetedMnliNoLogitsObjective, mnli, label_map, mnli_test[:20], "mnli_homoglyphs_targeted_nologits.pkl", maxiter=3)

#####Reordering Experiment

In [ ]:
mnli_targeted_experiment(ReorderTargetedMnliNoLogitsObjective, mnli, label_map, mnli_test[:20], "mnli_reorderings_targeted_nologits.pkl", maxiter=3)

#####Deletion Exeriment

In [ ]:
mnli_targeted_experiment(DeletionTargetedMnliNoLogitsObjective, mnli, label_map, mnli_test[:20], "mnli_deletions_targeted_nologits.pkl", maxiter=3)

## Availability Attacks

Sponge examples can be crafted from imperceptible perturbations. These examples are optimized to maximize inference compute time. When sent in large batches to ML systems, these examples cam be used to mount an ML denial of service (DoS) availability attack.

### Machine Translation Attacks

####Attack Setup

In [43]:
from timeit import timeit

class SpongeObjective(Objective):

  def objective(self) -> Callable[[List[float]], float]:
    def _objective(perturbations: List[float]) -> float:
      candidate: str = self.candidate(perturbations)
      return -1 * timeit(lambda: self.model.translate(candidate), number=1)
    return _objective

####Invsible Character Attack

In [46]:
class InvisibleCharacterSpongeObjective(SpongeObjective, InvisibleCharacterObjective):
  pass

#### Homoglyph Attack

In [44]:
class HomoglyphSpongeObjective(SpongeObjective, HomoglyphObjective):
  pass

####Reordering Attack

In [47]:
class ReorderSpongeObjective(SpongeObjective, ReorderObjective):
  pass

####Deletion Attack

In [48]:
class DeletionSpongeObjective(SpongeObjective, DeletionObjective):
  pass

#### Attack Performance

#####Invisible Character Experiment

In [ ]:
experiment(en2fr, InvisibleCharacterSpongeObjective, source_small, perturbs, 1, 6, 'invisibles_sponge.pkl', 3, 16)

##### Homoglyph Experiment

In [ ]:
experiment(en2fr, HomoglyphSpongeObjective, source_small, perturbs, 1, 6, 'homoglyph_sponge.pkl', 3, 16)

#####Reordering Experiment

In [ ]:
experiment(en2fr, ReorderSpongeObjective, source_small, perturbs, 1, 6, 'reorder_sponge.pkl', 3, 16)

####Deletion Experiment

In [ ]:
experiment(en2fr, DeletionSpongeObjective, source_small, perturbs, 1, 6, 'deletion_sponge.pkl', 3, 16)